In [1]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.tuner import IntegerParameter, ContinuousParameter, CategoricalParameter, HyperparameterTuner
from sagemaker import get_execution_role
import os

print("--- ⚙️ Step 1: Initializing SageMaker Environment ---")

# 1. 获取执行角色 (IAM Role)
# 如果在本地 VS Code 运行，可能需要手动指定 Role ARN
try:
    role = get_execution_role()
except ValueError:
    print("⚠️ 警告: 无法自动获取角色，请手动填入 ARN。")
    # 请去 AWS Console -> IAM -> Roles 找一个类似 AmazonSageMaker-ExecutionRole 的角色
    role = "arn:aws:iam::137568342316:role/SageMakerExecutionRole" 

# 2. 基础配置
sagemaker_session = sagemaker.Session()
bucket_name = 'sleep-disorder-mlops-bucket' # 您的 S3 桶名

# 3. 指定代码源 (Source of Truth for Code)
# SageMaker 会自动 `git clone` 这个仓库到训练实例中
git_repo = 'https://github.com/vinnie071015/sleeping-disorder-mlops.git' # 替换为您的仓库地址
git_config = {
    'repo': git_repo, 
    'branch': 'main'
}

print(f"✅ Role: {role}")
print(f"✅ Bucket: {bucket_name}")
print(f"✅ Git Repo: {git_repo}")

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/zhengchengsheng/Library/Application Support/sagemaker/config.yaml
--- ⚙️ Step 1: Initializing SageMaker Environment ---


Couldn't call 'get_role' to get Role ARN from role name ML_Project to get Role path.


⚠️ 警告: 无法自动获取角色，请手动填入 ARN。
✅ Role: arn:aws:iam::137568342316:role/SageMakerExecutionRole
✅ Bucket: sleep-disorder-mlops-bucket
✅ Git Repo: https://github.com/vinnie071015/sleeping-disorder-mlops.git


In [2]:
print("\n--- 💾 Step 2: Defining S3 Data Input ---")

# 定义 S3 数据输入
# SageMaker 会自动把这个 S3 路径下的所有文件下载到容器内的 /opt/ml/input/data/train/
# 这里的 s3_data 必须指向包含 sleep_data.csv 的文件夹路径 (以 / 结尾)
s3_input_path = f's3://{bucket_name}/raw_data/'

train_input = sagemaker.inputs.TrainingInput(
    s3_data=s3_input_path, 
    content_type='text/csv'
)

print(f"✅ Training Data Source: {s3_input_path}")


--- 💾 Step 2: Defining S3 Data Input ---
✅ Training Data Source: s3://sleep-disorder-mlops-bucket/raw_data/


In [7]:
print("\n--- 🏗️ Step 3: Defining Common Estimator Configuration (Git Mode) ---")

# 使用 ml.m5.large (通用型)
instance_type = 'ml.m5.large'

common_estimator_args = {
    # 修正 1: 入口文件路径要包含 src (因为 source_dir 变成了根目录)
    'entry_point': 'src/train.py',
    
    # 修正 2: source_dir 设为 '.' (代表 Git 仓库的根目录)
    # 这样 SageMaker 能同时看到 src/ 文件夹和 requirements.txt
    'source_dir': '.',
    
    'role': role,
    'instance_count': 1,
    'instance_type': instance_type,
    'framework_version': '1.2-1',
    'py_version': 'py3',
    'git_config': git_config,
    'sagemaker_session': sagemaker_session,
    
    # 修正 3: 不需要手动指定 dependencies 了
    # 因为 requirements.txt 就在 source_dir (根目录) 下，SageMaker 会自动识别并安装
    # 'dependencies': [...] <--- 删除这行
}

print(f"✅ Instance Type: {instance_type}")
print(f"✅ Path Correction: source_dir='.', entry_point='src/train.py'")


--- 🏗️ Step 3: Defining Common Estimator Configuration (Git Mode) ---
✅ Instance Type: ml.m5.large
✅ Path Correction: source_dir='.', entry_point='src/train.py'


In [8]:
print("\n--- 🎛️ Step 4: Defining Tuners for 3 Models ---")

# 定义指标抓取规则 (对应 src/train.py 中的 print 语句)
metric_definitions = [
    {'Name': 'accuracy', 'Regex': '✅ Accuracy: ([0-9\\.]+)'},
    {'Name': 'f1', 'Regex': '✅ F1 Score: ([0-9\\.]+)'}
]

# ==========================================
# A. Random Forest Tuner
# ==========================================
rf_estimator = SKLearn(**common_estimator_args)
# 固定模型类型为 RF
rf_estimator.set_hyperparameters(model_type='random_forest')

rf_tuner = HyperparameterTuner(
    estimator=rf_estimator,
    objective_metric_name='accuracy',
    metric_definitions=metric_definitions,
    hyperparameter_ranges={
        'n_estimators': IntegerParameter(50, 150),
        'max_depth': IntegerParameter(5, 15)
    },
    max_jobs=2,          # 总共跑 2 次 (省钱)
    max_parallel_jobs=1, # 串行跑 (安全)
    base_tuning_job_name='rf-tuning'
)

# ==========================================
# B. SVM Tuner
# ==========================================
svm_estimator = SKLearn(**common_estimator_args)
svm_estimator.set_hyperparameters(model_type='svm')

svm_tuner = HyperparameterTuner(
    estimator=svm_estimator,
    objective_metric_name='accuracy',
    metric_definitions=metric_definitions,
    hyperparameter_ranges={
        'C': ContinuousParameter(0.1, 5.0),
        'kernel': CategoricalParameter(['rbf', 'linear'])
    },
    max_jobs=2,
    max_parallel_jobs=1,
    base_tuning_job_name='svm-tuning'
)

# ==========================================
# C. Logistic Regression Tuner
# ==========================================
lr_estimator = SKLearn(**common_estimator_args)
lr_estimator.set_hyperparameters(model_type='logistic_regression')

lr_tuner = HyperparameterTuner(
    estimator=lr_estimator,
    objective_metric_name='accuracy',
    metric_definitions=metric_definitions,
    hyperparameter_ranges={
        'C': ContinuousParameter(0.1, 5.0)
    },
    max_jobs=2,
    max_parallel_jobs=1,
    base_tuning_job_name='lr-tuning'
)

print("✅ Tuners for RF, SVM, and LR are ready.")


--- 🎛️ Step 4: Defining Tuners for 3 Models ---
✅ Tuners for RF, SVM, and LR are ready.


In [9]:
import time

print("\n--- 🚀 Step 5: Launching Jobs Sequentially ---")

# 1. 运行 Random Forest
print(f"\n🌲 [1/3] Launching Random Forest Tuning... (Timestamp: {time.strftime('%H:%M:%S')})")
# SageMaker 会自动生成唯一的 Job Name，这里 fit 会直接打印出来
rf_tuner.fit({'train': train_input}, wait=True) 
print("✅ Random Forest Tuning Completed!")

# 2. 运行 SVM
print(f"\n🛡️ [2/3] Launching SVM Tuning... (Timestamp: {time.strftime('%H:%M:%S')})")
svm_tuner.fit({'train': train_input}, wait=True)
print("✅ SVM Tuning Completed!")

# 3. 运行 Logistic Regression
print(f"\n📈 [3/3] Launching Logistic Regression Tuning... (Timestamp: {time.strftime('%H:%M:%S')})")
lr_tuner.fit({'train': train_input}, wait=True)
print("✅ Logistic Regression Tuning Completed!")

print("\n🎉 All experiments finished!")


--- 🚀 Step 5: Launching Jobs Sequentially ---

🌲 [1/3] Launching Random Forest Tuning... (Timestamp: 21:51:39)


Cloning into '/var/folders/py/wsw9dv2n5g70rl8sm3pxxjg40000gn/T/tmp9ftsczkz'...
Already on 'main'


Your branch is up to date with 'origin/main'.


No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


.......................................................*


UnexpectedStatusException: Error for HyperParameterTuning job rf-tuning-251201-2151: Failed. Reason: All training jobs failed. Please take a look at the training jobs failures to get more details.. Check troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html

In [10]:
import boto3
import time

def diagnose_tuning_job(tuning_job_name=None):
    """
    全能诊断函数：结合了 API 状态查询和 CloudWatch 日志抓取。
    """
    # 如果没有传入名字，尝试从 rf_tuner 获取
    if tuning_job_name is None:
        try:
            tuning_job_name = rf_tuner.latest_tuning_job.name
        except NameError:
            print("❌ 错误: 未传入任务名，且内存中找不到 rf_tuner 对象。")
            return

    print(f"🕵️‍♂️ 正在全面诊断任务: {tuning_job_name}")
    
    sm_client = boto3.client('sagemaker')
    logs_client = boto3.client('logs')
    
    # 1. 列出该调优任务下的子训练任务
    try:
        tuner_resp = sm_client.list_training_jobs_for_hyper_parameter_tuning_job(
            HyperParameterTuningJobName=tuning_job_name,
            MaxResults=10,
            SortBy='CreationTime',
            SortOrder='Descending'
        )
    except Exception as e:
        print(f"❌ 无法找到任务或权限不足: {e}")
        return

    jobs = tuner_resp.get('TrainingJobSummaries', [])
    print(f"🔍 扫描了最近 {len(jobs)} 个子任务...\n")
    
    found_failure = False
    
    # 2. 遍历寻找失败的任务
    for job in jobs:
        job_name = job['TrainingJobName']
        status = job['TrainingJobStatus']
        
        print(f"➡️ 子任务: {job_name} | 状态: {status}")
        
        if status == 'Failed':
            found_failure = True
            print(f"\n🚨 锁定首个失败子任务: {job_name}")
            
            # --- A. 获取简短原因 (SageMaker API) ---
            detail = sm_client.describe_training_job(TrainingJobName=job_name)
            failure_reason = detail.get('FailureReason', '未知错误')
            
            print(f"\n❌ 核心报错简述 (FailureReason):\n{'-'*40}\n{failure_reason}\n{'-'*40}")
            
            # 智能建议
            if "AccessDenied" in failure_reason or "403" in failure_reason:
                print("💡 建议: IAM 角色权限不足，请检查是否附加了 AmazonS3FullAccess。")
            elif "ModuleNotFoundError" in failure_reason:
                print("💡 建议: 缺少依赖库。请检查 src/requirements.txt 是否包含该库，且已推送到 Git。")
            elif "FileNotFoundError" in failure_reason:
                print("💡 建议: S3 数据路径错误或文件名不匹配。请检查 Notebook 中的 Input Data 配置。")
            elif "ExecuteUserScriptError" in failure_reason:
                print("💡 建议: 代码在启动阶段崩溃。可能是依赖冲突或 import 错误。")
            
            # --- B. 获取详细日志 (CloudWatch Logs) ---
            # 只有当简短原因看不出问题（通常是 AlgorithmError/ExecuteUserScriptError）时，才需要看日志
            print(f"\n📝 正在尝试拉取 CloudWatch 详细崩溃日志...")
            
            try:
                # 获取日志流名称
                streams = logs_client.describe_log_streams(
                    logGroupName='/aws/sagemaker/TrainingJobs',
                    logStreamNamePrefix=job_name
                )
                
                if not streams['logStreams']:
                    print("⚠️ 警告: 该任务没有生成任何日志（可能在环境准备阶段就挂了，如依赖安装失败）。")
                else:
                    stream_name = streams['logStreams'][0]['logStreamName']
                    # 拉取最后 20 行日志
                    logs = logs_client.get_log_events(
                        logGroupName='/aws/sagemaker/TrainingJobs',
                        logStreamName=stream_name,
                        startFromHead=False,
                        limit=25
                    )
                    
                    print("\n" + "="*20 + " ☠️ 崩溃现场还原 (Traceback) ☠️ " + "="*20)
                    for event in logs['events']:
                        print(event['message'].rstrip())
                    print("="*65)
            
            except Exception as e:
                print(f"⚠️ 无法获取日志 (可能是日志组不存在): {e}")
            
            break # 只诊断最新的一个错误即可

    if not found_failure:
        print("\n✅ 当前扫描的子任务中没有发现 'Failed' 状态。")

# --- 使用方法 ---
# 直接运行即可，它会自动找 rf_tuner 的最新任务
if 'rf_tuner' in globals():
    diagnose_tuning_job(rf_tuner.latest_tuning_job.name)
else:
    print("rf_tuner 对象不存在，请手动传入任务名，例如: diagnose_tuning_job('rf-tuning-xxxx')")

🕵️‍♂️ 正在全面诊断任务: rf-tuning-251201-2151


/Users/zhengchengsheng/opt/anaconda3/envs/mlops-env/lib/python3.9/site-packages/boto3/compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


🔍 扫描了最近 2 个子任务...

➡️ 子任务: rf-tuning-251201-2151-002-b7b6db5b | 状态: Failed

🚨 锁定首个失败子任务: rf-tuning-251201-2151-002-b7b6db5b

❌ 核心报错简述 (FailureReason):
----------------------------------------
AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.9/site-packages/sagemaker_containers/_trainer.py", line 84, in train
    entrypoint()
  File "/miniconda3/lib/python3.9/site-packages/sagemaker_sklearn_container/training.py", line 39, in main
    train(environment.Environment())
  File "/miniconda3/lib/python3.9/site-packages/sagemaker_sklearn_container/training.py", line 31, in train
    entry_point.run(uri=training_environment.module_dir,
  File "/miniconda3/lib/python3.9/site-packages/sagemaker_training/entry_point.py", line 108, in run
    return runner.get(runner_type, user_entry_point, args, env_vars, extra_opts).run(
  File "/miniconda3/lib/python3.9/site-packages/sagemaker_training/process.py", line 424, in run
    process = check_error(
 